# SPSS Modelerの再構成ノードをPythonで書き換える。購入商品カテゴリごとの集計


■加工前<BR>
誰（CUSTID）がいつ（SDATE）何（PRODUCTID、L_CLASS商品大分類、M_CLASS商品中分類）をいくら（SUBTOTAL）購入したかが記録されたID付POSデータを使います。

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/1da09263-06f2-ecee-b099-cfaedb28a0a6.png)

■加工後<BR>
顧客毎(CUSTID)に商品大分類（L_CLASS）の①商品カテゴリごとの購入額合計と②商品カテゴリごとの購入割合を集計します。
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/ecae01c9-9fe3-cd25-8daf-82b8df04778c.png)



In [1]:
!wget https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/sampletranDEPT4en2019S.csv

--2020-06-30 12:30:39--  https://raw.githubusercontent.com/hkwd/200611Modeler2Python/master/data/sampletranDEPT4en2019S.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.228.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.228.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1463149 (1.4M) [text/plain]
Saving to: ‘sampletranDEPT4en2019S.csv.5’

100%[======================================>] 1,463,149   --.-K/s   in 0.05s   

2020-06-30 12:30:40 (28.1 MB/s) - ‘sampletranDEPT4en2019S.csv.5’ saved [1463149/1463149]



In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

import pandas as pd
df = pd.read_csv('sampletranDEPT4en2019S.csv')

In [3]:
pd.options.display.max_rows = 10
print(pd.options.display.max_rows)

10


In [4]:
df.head(15)

,CUSTID,SDATE,PRODUCTID,L_CLASS,M_CLASS,SUBTOTAL
0,100001,2019-07-02,9937845,BAG,BAG01,1740
1,100001,2019-07-09,9903878,BAG,BAG01,3108
2,100001,2019-08-08,9922209,COSMETICS,COSMETICS03,2888
3,100001,2019-08-18,9901187,SHOES,SHOES02,2700
4,100001,2019-08-26,9910507,COSMETICS,COSMETICS03,3385
...,...,...,...,...,...,...
10,100001,2019-11-28,9900330,SHOES,SHOES02,1672
11,100004,2019-01-16,9905063,SHOES,SHOES03,12850
12,100004,2019-05-14,9903627,COSMETICS,COSMETICS03,4076
13,100004,2019-06-25,9910983,SHOES,SHOES03,23000


# ①商品大分類ごとの合計購入額

再構成ノードと集計ノード、置換ノード、集計ノード、レコード結合ノードの書き換え
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/d50deec3-cacd-1032-0bf1-14a4ac2b0398.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/17f26887-1111-969f-daee-0a9907615191.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/0e88a1ed-23c4-619c-d595-91244e6285fc.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/dcd1a4b1-dbfd-cbf6-43cc-aa86ee49717d.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/22071bf7-b930-f105-7b56-9d63de9af94f.png)
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/8bf5146a-5434-b635-e2b3-92facaa08554.png)

Float values when pivoting dataframe of integers (pandas - python) - Stack Overflow
https://stackoverflow.com/questions/50351530/float-values-when-pivoting-dataframe-of-integers-pandas-python

pandasのピボットテーブルでカテゴリ毎の統計量などを算出 | note.nkmk.me
https://note.nkmk.me/python-pandas-pivot-table/

In [5]:
#再構成ノードと集計ノード、置換ノード、集計ノード、レコード結合ノードの組合せはpivot_tableで置き換えられる
#すべての大分類が自動的に対象になるので、集計対象を絞り込みたい場合はあらかじめ絞り込む必要がある。
#fill_value=0でfillna(0)の処理が行われる
#fill_value=0,aggfunc='sum'を入れないとFloat変換されてしまう。
#margins=True, margins_name='SUBTOTAL'で列合計、行合計の列を追加できる
pivot_tran_df= df.pivot_table(
    index=['CUSTID'],columns=['L_CLASS'],values=['SUBTOTAL'],
    aggfunc='sum',
    fill_value=0,
    margins=True, margins_name='ALL')
pivot_tran_df

SUBTOTAL                               
L_CLASS       BAG COSMETICS     SHOES        ALL
CUSTID                                          
100001       7243     10273     26870      44386
100004       2738     14987     45890      63615
100006       4600      3600         0       8200
100008       2800     11028     11123      24951
100012      60782      3560      8524      72866
...           ...       ...       ...        ...
105970       3657    109837    132122     245616
105972          0      2964     35780      38744
105974      12930     75005     54602     142537
105978          0      1596     23851      25447
ALL      29552923  47559347  57304999  134417269

[2745 rows x 4 columns]

python — pandasデータフレームのデータの最後の行を削除する方法
https://www.it-swarm.dev/ja/python/pandas%E3%83%87%E3%83%BC%E3%82%BF%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E6%9C%80%E5%BE%8C%E3%81%AE%E8%A1%8C%E3%82%92%E5%89%8A%E9%99%A4%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95/1050572731/

In [6]:
#行合計はいらないので削除
pivot_tran_df=pivot_tran_df[:-1]
pivot_tran_df

SUBTOTAL                          
L_CLASS      BAG COSMETICS   SHOES     ALL
CUSTID                                    
100001      7243     10273   26870   44386
100004      2738     14987   45890   63615
100006      4600      3600       0    8200
100008      2800     11028   11123   24951
100012     60782      3560    8524   72866
...          ...       ...     ...     ...
105967     57800      6980       0   64780
105970      3657    109837  132122  245616
105972         0      2964   35780   38744
105974     12930     75005   54602  142537
105978         0      1596   23851   25447

[2744 rows x 4 columns]

In [7]:
#pivot_tableをつかうとカラムがマルチカラムになる
pivot_tran_df.columns

MultiIndex(levels=[['SUBTOTAL'], ['ALL', 'BAG', 'COSMETICS', 'SHOES']],
           codes=[[0, 0, 0, 0], [1, 2, 3, 0]],
           names=[None, 'L_CLASS'])

In [8]:
#マルチカラムのままだと結合などがしづらく扱いづらいので、フラットなスネーク形式（アンダースコアで連結）に変換する
pivot_tran_df.columns = [
    pivot_tran_df.columns.names[1]+"_"+levels[1]+"_"+levels[0] 
    for levels in pivot_tran_df.columns]
pivot_tran_df.head()

,L_CLASS_BAG_SUBTOTAL,L_CLASS_COSMETICS_SUBTOTAL,L_CLASS_SHOES_SUBTOTAL,L_CLASS_ALL_SUBTOTAL
CUSTID,,,,
100001,7243,10273,26870,44386
100004,2738,14987,45890,63615
100006,4600,3600,0,8200
100008,2800,11028,11123,24951
100012,60782,3560,8524,72866


In [9]:
#フラットになったカラム
pivot_tran_df.columns

Index(['L_CLASS_BAG_SUBTOTAL', 'L_CLASS_COSMETICS_SUBTOTAL',
       'L_CLASS_SHOES_SUBTOTAL', 'L_CLASS_ALL_SUBTOTAL'],
      dtype='object')

フィルターノードの置き換え
![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/557dc134-cbf6-43f4-895f-4208bb80039a.png)

pandas.DataFrameの行名・列名の変更 | note.nkmk.me https://note.nkmk.me/python-pandas-dataframe-rename/

Pythonで文字列を置換（replace, translate, re.sub, re.subn） | note.nkmk.me
https://note.nkmk.me/python-str-replace-translate-re-sub/

In [10]:
#正規表現を使って列名変更
import re
pivot_tran_df=pivot_tran_df.rename(
    columns= lambda str:
    re.sub('L_CLASS_(.+)_SUBTOTAL',r'\1_SUM',str))
pivot_tran_df.head()

,BAG_SUM,COSMETICS_SUM,SHOES_SUM,ALL_SUM
CUSTID,,,,
100001,7243,10273,26870,44386
100004,2738,14987,45890,63615
100006,4600,3600,0,8200
100008,2800,11028,11123,24951
100012,60782,3560,8524,72866


# ②各商品大分類購入の割合



フィールド追加ノードの置き換え![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/2e8aae3e-15b5-4ceb-1887-77efebc8e995.png)

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/223763/e0b5f7c3-6915-45e5-9279-9c66f2e00e43.png)

pandas.DataFrame.div — pandas 0.24.2 documentation
https://pandas.pydata.org/pandas-docs/version/0.24.2/reference/api/pandas.DataFrame.div.html

python - データフレームの複数列を一つの列で割る - スタック・オーバーフロー
https://ja.stackoverflow.com/questions/52834/%E3%83%87%E3%83%BC%E3%82%BF%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E3%81%AE%E8%A4%87%E6%95%B0%E5%88%97%E3%82%92%E4%B8%80%E3%81%A4%E3%81%AE%E5%88%97%E3%81%A7%E5%89%B2%E3%82%8B

In [11]:
#divで全列に割り算ができる。
pivot_tran_ratio_df=pivot_tran_df.div(pivot_tran_df["ALL_SUM"], axis=0)
pivot_tran_ratio_df.head()

,BAG_SUM,COSMETICS_SUM,SHOES_SUM,ALL_SUM
CUSTID,,,,
100001,0.163182,0.231447,0.605371,1.0
100004,0.043040,0.235589,0.721371,1.0
100006,0.560976,0.439024,0.000000,1.0
100008,0.112220,0.441986,0.445794,1.0
100012,0.834161,0.048857,0.116982,1.0


In [12]:
#既存のDataFrameが置き換えられるので、列名を変更し、ALL列は削除する
import re
pivot_tran_ratio_df=pivot_tran_ratio_df\
    .drop(columns=['ALL_SUM'])\
    .rename(columns= lambda str:re.sub('_SUM','_RATIO',str))
pivot_tran_ratio_df.head()

,BAG_RATIO,COSMETICS_RATIO,SHOES_RATIO
CUSTID,,,
100001,0.163182,0.231447,0.605371
100004,0.043040,0.235589,0.721371
100006,0.560976,0.439024,0.000000
100008,0.112220,0.441986,0.445794
100012,0.834161,0.048857,0.116982


In [13]:
#①商品大分類ごとの合計購入額のDataFrameに結合しなおす。
pivot_tran_df=pivot_tran_df.join(pivot_tran_ratio_df)
pivot_tran_df.head()

,BAG_SUM,COSMETICS_SUM,SHOES_SUM,ALL_SUM,BAG_RATIO,COSMETICS_RATIO,SHOES_RATIO
CUSTID,,,,,,,
100001,7243,10273,26870,44386,0.163182,0.231447,0.605371
100004,2738,14987,45890,63615,0.043040,0.235589,0.721371
100006,4600,3600,0,8200,0.560976,0.439024,0.000000
100008,2800,11028,11123,24951,0.112220,0.441986,0.445794
100012,60782,3560,8524,72866,0.834161,0.048857,0.116982


In [14]:
import sys

print(sys.version)

3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]


In [15]:
print(pd.__version__)

0.24.1
